In [11]:
import farter_config as cfg

config_loader = cfg.ConfigLoader()
model_config = config_loader.load('nvidia_v1')
print(model_config)

ModelConfig(raw_input=[RawInputConfig(ticker='NVDA', resolution=<CandleResolution.THIRTY_MINUTES: 3>, first_date=datetime.date(2021, 1, 1), last_date=datetime.date(2021, 2, 1))])
